# Rozdzial 5.3


5.3 Nowy atak kryptoanalityczny z tekstu zaszyfrowanego
Nowy atak został opracowany zgodnie z naszą nową metodologią, opisaną w Rozdziale 4. Polega on na sekwencyjnym algorytmie wspinaczkowym dwufazowym (patrz Sekcja 4.3) z wielokrotnymi restartami, zaprojektowanym w celu wytworzenia wysokiej jakości kluczy początkowych dla głównego wyszukiwania (patrz Sekcja 4.7). Wprowadza on adaptacyjną ocenę, wykorzystując nowe specjalistyczne metody oceny, aby osiągnąć większą odporność na błędy w pierwszej fazie (patrz Sekcja 4.5), a także nowe typy transformacji zaprojektowane w celu poprawy pokrycia wyszukiwania (patrz Sekcja 4.6).

Zaczynamy w Sekcji 5.3.1 od opisu podstawowego algorytmu wspinaczkowego, który jest podobny do kilku algorytmów używanych w poprzednich pracach. Celem tego algorytmu bazowego jest służenie jako podstawa dla dalszych ulepszeń, a także jako punkt odniesienia dla oceny wydajności.

W Sekcji 5.3.2 stopniowo poprawiamy ten podstawowy algorytm. Przedstawiamy nowe typy transformacji klucza niedestrukcyjnych - przesuwania i zamiany segmentów, dostosowanych do szyfru transpozycji kolumnowej. Te nowe transformacje znacząco zwiększają pokrycie wyszukiwania. Ta wersja algorytmu jest skuteczna z kluczami średniej długości (CCT) oraz krótkimi kluczami w przypadku ICT.

W Sekcji 5.3.3 dodajemy fazę początkową, aby wytworzyć klucze początkowe wysokiej jakości dla głównej fazy. Ta początkowa faza wykorzystuje nową specjalistyczną funkcję oceny - ocenę sąsiedztwa. Ta wersja algorytmu jest w stanie rozwiązać przypadki CCT z długimi kluczami.

W Sekcji 5.3.4 prezentujemy najbardziej złożoną wersję algorytmu, przeznaczoną do rozwiązywania przypadków Incomplete Columnar Transposition (ICT) z długimi kluczami. Ten algorytm wspinaczkowy również ma dwie fazy, początkową fazę wykorzystującą nową specjalistyczną funkcję oceny - ocenę wyrównania, oprócz zmodyfikowanej oceny sąsiedztwa.

W każdej sekcji analizujemy wydajność odpowiednich algorytmów i ich ulepszeń, dla kilku scenariuszy CCT i ICT, obejmujących różne wartości |K| i r.
Wszystkie metody opisane w tym rozdziale zakładają, że długość klucza transpozycji, |K|, jest znana. Jeśli nie jest znana, algorytmy muszą być uruchamiane wielokrotnie dla każdej możliwej długości klucza.
5.3.1 Bazowy algorytm wspinaczkowy
Zaimplementowaliśmy bazowy algorytm wspinaczkowy, który służy nam jako odniesienie w porównaniach wydajności oraz jako podstawa dla opisanych później usprawnień. Implementacja bazowego algorytmu wspinaczkowego składa się z następujących kroków:

1.	Wygeneruj losowy klucz transpozycji.
2. Dla każdej możliwej transformacji klucza:
(a) Zastosuj transformację do bieżącego klucza i otrzymaj nowy klucz kandydujący.
(b) Odszyfruj szyfrogram za pomocą klucza kandydującego, aby uzyskać rzekomy tekst jawnie.
(c) Oceń rzekomy tekst jawnie na podstawie funkcji oceny.
(d) Jeśli ocena jest wyższa niż przed transformacją, zachowaj klucz kandydujący jako bieżący klucz.
3. Jeśli nie można już uzyskać poprawy oceny przez transformacje klucza, albo znaleźliśmy poprawny klucz (i kończymy), albo osiągnęliśmy lokalne maksimum, w którym przypadku restartujemy cały proces od kroku 1.
Transformacje klucza polegają na zamianie dwóch elementów klucza. Ten typ transformacji jest również stosowany w większości publikacji wcześniej wspomnianych w bibliografii. Wszystkie możliwe zamiany dwóch elementów klucza są sprawdzane w każdej iteracji wspinaczki, co daje łącznie |K| ·(|K| − 1) ÷ 2 opcji. Jako funkcję oceny używamy sumy logarytmów częstotliwości quadgramów (4 kolejnych znaków) w rzekomym tekście jawnym.
W Tabeli 5.5 podsumowujemy wyniki uzyskane tym bazowym algorytmem, stosując proste zamiany i pojedynczą fazę. Wydajność tej implementacji bazowej jest podobna do wyników uzyskanych za pomocą metod opracowanych w wcześniejszych pracach. We wszystkich przypadkach prostokąty transpozycji są kompletne (przypadek CCT).
5.3.2 Ulepszony algorytm dla kluczy średniej długości
Pierwsza poprawka ma na celu umożliwienie algorytmowi pracy z krótszymi szyfrogramami oraz obsługę kluczy o średniej długości (do |K| = 100) w przypadku CCT oraz krótkich kluczy (do |K| = 30) w przypadku ICT. Osiągnięto to poprzez wprowadzenie nowych typów transformacji, opisanych poniżej.
Początkowo eksperymentowaliśmy z wykorzystaniem nowego typu transformacji, Segment Swap. Ta transformacja polega na zamianie dwóch segmentów kolejnych elementów klucza. Podczas przeszukiwania algorytmem wzgórzowym, sprawdzamy wszystkie możliwe transformacje Segment Swap dla każdej możliwej pozycji początkowej p1 i p2 dwóch segmentów, dla każdej wielkości segmentu l, pod warunkiem, że segmenty się nie nakładają. W poniższym przykładzie klucz to ABCDEFGHIJ. Podkreślone litery reprezentują dwa segmenty o długości l = 2, które chcemy zamienić, "CD" i "HI", począwszy od pozycji p1 = 3 i p2 = 8, odpowiednio. Po zamianie segmentów otrzymujemy następujący klucz: ABHIEFGCDJ.
Zmodyfikowaliśmy bazowy algorytm, aby używał zamiany segmentów zamiast pojedynczych zamian, ale poprawa była marginalna i prawie niezauważalna. Następnie spróbowaliśmy nowego typu transformacji, Segment Slide (można też opisać jako cykliczna rotacja lub przesunięcie). Przy tej transformacji wybieramy segment o długości l, począwszy od pozycji p, i przesuwamy ten segment w prawo, s razy. W poniższym przykładzie klucz to ABCDEFGHIJ, a podkreślone litery reprezentują segment, który chcemy przesunąć 3 razy w prawo (l = 3, p = 3, s = 3). Po przeprowadzeniu transformacji Segment Slide otrzymujemy następujący klucz: ABFGHCDEIJ. Podkreślone litery są literami, na które wpłynęła transformacja. Należy zauważyć, że podczas przesuwania segmentu "CDE", segment "FGH" również został przesunięty.
Ważną cechą transformacji wykonywanych na segmentach (tj. segmentowe przesunięcia i segmentowe zamiany) jest to, że - jak sama nazwa wskazuje - działają one na sąsiadujących segmentach elementów klucza. Dlatego większość relacji przyległości, tj. który kolumna j znajduje się bezpośrednio na prawo od innej kolumny i, jest zachowana.
Wykorzystując ulepszony algorytm, można w pełni odzyskać klucze CCT o długości do co najmniej 100 elementów. Poprzednie prace i algorytm bazowy nie były w stanie odzyskać kluczy dłuższych niż 30. Testy przeprowadzone na kluczach o jeszcze większych długościach, sięgających 200, również okazały się udane, ale dla kluczy dłuższych niż 100 algorytm może odnieść sukces, ale wymaga to bardzo dużo czasu, ponieważ liczba transformacji do sprawdzenia jest proporcjonalna do |K|3. Oznacza to, że przy |K|=100, w każdym kroku procesu hill climbing musi zostać sprawdzonych około 1 413 751 ≈ 221 unikalnych transformacji, zgodnie z naszymi symulacjami. W następnej sekcji (5.3.3) przedstawiamy dalsze optymalizacje, mające na celu nie tylko zwiększenie prawdopodobieństwa sukcesu, ale także przyspieszenie procesu odzyskiwania klucza w przypadku CCT o bardzo długich kluczach.
W przypadku ICT (szyfru tylko-zaszyfrowanego) okazuje się, że ten algorytm działa skutecznie na krótkich kluczach (długość 15-20), mniej skutecznie na kluczach o długości 25-30, gdyż wymaga to bardzo długiego tekstu szyfrogramu, a dla kluczy o długości 50 jest praktycznie nieskuteczny i całkowicie nieskuteczny dla dłuższych kluczy. W sekcji 5.3.4 wprowadzono ulepszony algorytm, zdolny do odzyskiwania kluczy w przypadku ICT przy znacznie krótszych tekstach szyfrogramów oraz dłuższych kluczach ICT, do |K|=120.
W dalszej części oceny wykorzystywane są zarówno przesunięcia segmentowe, jak i zamiany segmentowe jako transformacje. Później wrócimy i ocenimy ich odpowiedni wkład w kontekście końcowych algorytmów.
Następne usprawnienie opisane w sekcji 5.3.3 ma na celu umożliwienie skutecznej kryptoanalizy w przypadku CCT z bardzo długimi kluczami oraz przyspieszenie algorytmu w ogóle. Jedną z głównych cech (i słabości) szyfru transpozycji kolumnowej jest to, że kolumny oryginalnego tekstu jawnego są zachowywane i pojawiają się jako ciągłe segmenty tekstu w szyfrogramie (patrz Rysunek 5.1). Algorytm opisany w tej sekcji wykorzystuje tę właściwość, polegając na relacjach między kolumnami szyfrogramu.
Pierwsza kolumna szyfrogramu może rozpoczynać się tylko na początku szyfrogramu, na pozycji 1. Druga kolumna może zaczynać się od pozycji r+1 lub r+2, w zależności od tego, czy pierwsza kolumna była krótka (o r elementach), czy długa (o r+1 elementach). Trzecia kolumna może zaczynać się od pozycji 2r+1, 2r+2 lub 2r+3, i tak dalej dla kolejnych kolumn. Dla kolumny pośrodku szyfrogramu może być do około |K|/2 możliwych pozycji początkowych. Aby obliczyć wynik sąsiedztwa dla dowolnej pary kolumn i i j, musimy najpierw poznać pozycje początkowe obu kolumn i lub przynajmniej względną różnicę lub przesunięcie modulo r między tymi pozycjami początkowymi, aby móc poprawnie wyrównać obie kolumny. Ale nie możemy osiągnąć tego, chyba że znamy klucz. Z powodu tej niejednoznaczności co do pozycji początkowych kolumn musimy dostosować obliczenie wyniku sąsiedztwa do przypadku ICT. Mimo że nie znamy pozycji początkowych kolumn w szyfrogramie, nadal możemy spróbować oszacować, na ile dobrze wyjustowane są pozycje początkowe kolumn w parach sąsiednich dla danego klucza kandydującego. W tym celu wprowadzamy nowy typ wyniku o nazwie "wynik wyrównania".

Modyfikacja wyniku sąsiedztwa: Aby dostosować wynik sąsiedztwa do ICT, musimy teraz obliczyć wynik sąsiedztwa dla kolumn (i, j), tak jak opisano w poprzedniej sekcji dla CCT, dla każdej kombinacji możliwych pozycji początkowych pi i pj dla tych kolumn. Zachowujemy najlepszy wynik sąsiedztwa uzyskany z najlepszą kombinacją pi i pj jako ostateczny wynik sąsiedztwa dla (i, j). Ponadto obliczamy różnicę między tymi optymalnymi pozycjami początkowymi pi i pj i zachowujemy ich różnicę modulo r w macierzy przesunięć O[i, j], z O[i, j]=(pj − pi) mod r, przy czym r jest liczbą pełnych wierszy. Należy zauważyć, że ta macierz przesunięć musi być obliczona tylko w przypadku ICT. W przypadku CCT wszystkie kolumny mają tę samą długość, a wszystkie przesunięcia modulo r są równe 0, dlatego ta macierz przesunięć jest nieistotna.

Aby zrozumieć, co oznacza ten nowy wynik, który ma być mierzony, należy ponownie przyjrzeć się przykładowi transpozycji przedstawionemu na rysunku 5.1. Długość klucza |K| wynosi 7, a liczba pełnych wierszy to r =   |C| |K|  =  45/7 = 6. Analizujemy tu dopasowanie kolumn 2 i 7 w szyfrogramie. Obie kolumny były sąsiadujące w oryginalnej planszy tekstu jawnego przed transpozycją. Oba są długimi kolumnami, o łącznej liczbie elementów r +1 = 7. Kolumna 2 składa się z ciągu liter HINIEPX, a kolumna 7 z ciągu ECSOROT. Kolumna 2 zaczyna się na pozycji 7 w szyfrogramie, a kolumna 7 na pozycji 33. Gdy zostaną połączone w sposób właściwy, tworzą bigramy HE, IC, NS, IO, ER, PO i XT, łącznie siedem bigramów, które pierwotnie występowały w tekście jawnym. Pokazano to na rysunku 5.5, część (1).
Jeśli nie znamy klucza, to jedyną rzeczą, jaką wiemy, jest to, że kolumna 2 mogła zacząć się od pozycji 7 lub 8, w zależności od tego, czy kolumna 1 była krótka czy długa. Kolumna 7 to ostatnia kolumna, mogła zacząć się od pozycji 33 lub 34, w zależności od tego, czy była długa czy krótka.
Jeśli jednak jesteśmy w stanie odgadnąć prawidłowe pozycje startowe kolumn 2 i 7, jak pokazano na rysunku 5.5, można odtworzyć wszystkie siedem oryginalnych bigramów. W tym przypadku różnica modulo r między tymi poprawnymi pozycjami startowymi wynosi  (33-7) mod 6 = 2.
Jeśli źle odgadniemy jedną z pozycji startowych i poprawnie odgadniemy drugą, to nie można odtworzyć żadnego z oryginalnych bigramów. W przykładzie ilustrowanym na rysunku 5.5, poprawnie odgadliśmy pozycję startową (33) kolumny 7, ale pozycja startowa (8) kolumny 2 była błędna. W tym przypadku różnica modulo r między pozycjami startowymi kolumn wynosi (33−8) mod 6 = 1, a wszystkie odtworzone bigramy są nieprawidłowe.
Do obliczenia wyniku wyrównania, używamy macierzy przesunięć O[i, j]. Wynik wyrównania obliczamy dla kandydata na klucz, przetwarzając wszystkie pary sąsiadujących kolumn i oraz j, w następujący sposób:

1.	Liczymy liczbę długich kolumn, według kandydata na klucz, między dwiema kolumnami i i j, włączając i samą kolumnę, ale wykluczając j.
2. Porównujemy tę liczbę z wartością O[i, j] (uzyskaną podczas obliczania wyniku sąsiedztwa).
(a) Jeśli są równe, dodajemy 2 punkty do wyniku wyrównania kandydata na klucz.
(b) Jeśli są równe, i dodatkowo wcześniej także dodaliśmy punkty dla poprzedniej pary sąsiadujących kolumn k i i, gdzie k to kolumna po lewej stronie od i, dodajemy dodatkowy punkt do wyniku wyrównania. Ma to na celu przypisanie wyższego wyniku do kolejnych sekwencji dobrze wyrównanych sąsiadujących par kolumn w kandydacie na klucz.
Następnie pokazujemy, jak w fazie 1 używamy zarówno wyniku sąsiedztwa, jak i wyniku wyrównania do generowania optymalnego klucza początkowego dla fazy 2. Oto ulepszony algorytm fazy 1 dla przypadku ICT:
3. Faza 1:
(a) Przedoblicz macierz wyniku sąsiedztwa dla wszystkich możliwych par kolumn i i j, takich jak 1 ≤ i ≤ |K|, 1 ≤ j ≤ |K|, i = j, przy użyciu zmodyfikowanej metody obliczania wyniku sąsiedztwa opisanej wcześniej w tej sekcji. W wyniku obliczana jest również macierz przesunięć O[i, j].
(b) Wygeneruj losowy początkowy kandydat na klucz.
(c) Ustaw początkowe minimalne progi dla wyniku sąsiedztwa i początkowe minimalne progi dla wyniku wyrównania.
(d) Wykonuj iteracyjnie następująco, do 15 razy (lub przerwij wcześniej, jeśli nie można było osiągnąć dalszej poprawy):
i. Iteracyjnie wykonaj następujące czynności dla każdej możliwej transformacji Segment Slide i Segment Swap na bieżącym kluczu:
A. Zastosuj transformację do bieżącego klucza, aby uzyskać kandydata na klucz.
B. Oblicz punktację pierwotną, punktację sąsiedztwa dla klucza kandydującego. Jeśli jest niższa niż wynik przed transformacją, odrzuć klucz kandydujący. W przeciwnym razie, oblicz także punktację drugorzędną, punktację wyrównania dla klucza kandydującego. Jeśli jest poniżej obecnego progu punktacji wyrównania, odrzuć klucz kandydujący. W przeciwnym razie, zachowaj klucz kandydujący jako nowy klucz bieżący.
ii. Wykonaj iteracyjnie następujące kroki dla każdej możliwej transformacji przesunięcia segmentu i wymiany segmentów na bieżącym kluczu. A. Zastosuj transformację do bieżącego klucza, aby uzyskać klucz kandydujący. B. Oblicz tym razem punktację pierwotną, czyli punktację wyrównania dla klucza kandydującego. Jeśli jest niższa niż wynik przed transformacją, odrzuć klucz kandydujący. W przeciwnym razie, oblicz także punktację drugorzędną, punktację sąsiedztwa dla klucza kandydującego. Jeśli jest poniżej obecnego progu punktacji sąsiedztwa, odrzuć klucz kandydujący. W przeciwnym razie, zachowaj klucz kandydujący jako nowy klucz bieżący.
iii. Zwiększ progi zarówno dla punktacji sąsiedztwa, jak i dla punktacji wyrównania.
(e) Użyj ostatniego klucza bieżącego jako klucza początkowego dla Fazy 2 i rozpocznij Fazę 2. Algorytm Fazy 2 jest taki sam jak w Sekcji 5.3.3. Celem tego algorytmu Fazy 1 jest maksymalizacja zarówno punktacji sąsiedztwa, jak i punktacji wyrównania. Moglibyśmy zamiast tego użyć liniowej kombinacji tych punktacji jako jednej funkcji celu, ale eksperymenty w tej kierunku okazały się bezowocne. Zamiast tego opracowaliśmy to dwuwymiarowe wyszukiwanie. Przy każdej głównej iteracji, zmieniamy metodę punktacji pierwotnej, używając jako punktacji do maksymalizacji punktacji sąsiedztwa lub punktacji wyrównania. Obliczamy także punktację drugorzędną (drugą punktację), ale tylko sprawdzamy, czy jest powyżej minimalnego progu. Dlatego punktacja drugorzędną często może się zmniejszać, podczas gdy punktacja pierwotna wzrasta, generując "falisty" wzór.
Jednak ogólna tendencja jest taka, że obie oceny rosną, gdy podnosimy progi dla ocen wtórnych po każdej iteracji. W praktyce po kilku iteracjach zjawisko zygzakowania zwykle ustaje, a obie oceny rosną razem. Ilustruje to rysunek 5.6. Oceniliśmy wydajność tego ulepszonego algorytmu dla kluczy o długości do 120 znaków, dla najgorszego przypadku ICT. Najgorszy przypadek dla ICT występuje, gdy liczba długich kolumn u jest równa liczbie krótkich kolumn lub jest mniej więcej równa |K|/2. W tabeli 5.10 pokazujemy minimalną długość szyfrogramu wymaganą do pełnego odzyskania klucza (i 100% skuteczność), dla kilku długości kluczy. Dla porównania, dla kluczy o długości |K| = 50, minimalna wymagana długość szyfrogramu wynosi tylko 825 w porównaniu do 2525 przy użyciu opisanego w sekcji 5.3.2 algorytmu (tabela 5.8), a skuteczność wynosi 100% w porównaniu do 70%. Ponadto, ten algorytm dobrze działa z dość długimi kluczami, aż do |K| = 120, jak pokazano w tabeli 5.10. Sporadyczne testy wykazały, że może działać nawet dla dłuższych kluczy. Analizowaliśmy również wydajność algorytmu w zależności od liczby długich kolumn u (rys. 5.7). Jak widać, najgorszy przypadek, wymagający najwięcej wierszy r (i dlatego najdłuższego szyfrogramu) do rozwiązania, to przypadek, gdy liczba długich kolumn wynosi u ≈ |K|/2. Należy zauważyć, że wydajność dla u ≈ |K|· 3/4, choć tutaj nie pokazana, jest podobna do przypadku u ≈ |K|/4. Wreszcie, oceniliśmy czas potrzebny na złamanie klucza dla ulepszonego algorytmu dla najgorszego przypadku ICT (u ≈ |K|/2). W tabeli 5.11 pokazujemy średnią liczbę deszyfracji wymaganych do pełnego odzyskania klucza dla różnych scenariuszy długości klucza ICT. Dla każdej długości zawieramy ekstremalny przypadek, z minimalną liczbą wierszy wymaganą do pełnego odzyskania klucza, oraz umiarkowany przypadek, z większą liczbą wierszy.
5.4 Podsumowanie
W poprzednich pracach nad komputerową kryptoanalizą klasycznej szyfru transpozycji kolumnowej, podejście polegało na bezpośrednim zastosowaniu jednego z kilku ogólnych algorytmów, takich jak wspinaczka po grze, algorytmy genetyczne lub symulowane wyżarzanie. To naiwne podejście, choć łatwe do zaimplementowania, miało poważne ograniczenia. Działało tylko dla krótkich kluczy, długich szyfrogramów i nie potrafiło skutecznie poradzić sobie z bardziej wymagającym przypadkiem ICT.
Praca przedstawiona tutaj demonstruje skuteczność metodyki kryptoanalizy szyfrów transpozycji kolumnowych z trudnymi ustawieniami, jak podsumowano w tabeli 5.12. Znacznie ulepszyliśmy podstawowy algorytm, stosując specjalistyczne transformacje (przesuwania segmentów), specjalistyczne funkcje oceny (oceny przyległości i wyrównania) oraz dodanie wstępnej fazy, która polega na głębokiej analizie relacji między kolumnami szyfrogramu. Nasza metoda jest bardzo skuteczna w kryptoanalizie szyfrów transpozycji kolumnowych, gdy są szyfrowane bardzo długie klucze, aż do |K| = 1 000 elementów dla przypadku CCT, a do |K| = 120 elementów dla bardziej złożonego przypadku ICT. Wymaga również znacznie mniej materiału szyfrogramu niż poprzednie metody. Na przykład, w przypadku CCT, dla klucza z |K| = 30 elementami, szyfrogram zaledwie z |C| = 180 literami można zdekodować kryptoanalitycznie przy użyciu tego nowego algorytmu, w porównaniu do co najmniej |C| = 990, czyli pięciokrotnie więcej, dla najbardziej wydajnych z innych poprzednich metod.